#### Aqui iniciei com o import do selenium
###### Mas antes foi preciso fazer a instalação das bibliotecas:
- pip install selenium
- pip install scrapy
- pip install ipython

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
import re

##### Fiz a instalação do chromedriver e em seguida abro a página do Google

In [ ]:
'''# arquivo csv
writer = csv.writer(open('Scraping - Francisco Davi Gomes de Oliveira.csv', 'w', encoding='utf-8'))
writer.writerow(['URL da vaga no linkedin', 'Nome da vaga', 
                 'Nome da empresa contratante','URL da empresa contratante','Modelo de contratação',
                'Tipo de contratação','Nível de experiência','Número de candidaturas para vaga','Data da postagem da vaga',
                 'Horário do scraping', 'Número de funcionários da empresa', 'Número de seguidores da empresa',
                'Local sede da empresa','URL da candidatura'])'''

In [ ]:

# Configurar o driver do navegador (exemplo usando o Chrome)
options = Options()
options.add_argument("start-maximized")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get("https://www.google.com")

##### Em seguida, insiro a palavra chave para busca, e abro o Linkedin

In [ ]:
driver.find_element('name', 'q').send_keys('vagas marketing')
driver.get("https://www.linkedin.com")

###### Ao abrir o Linkedin, não irei logar, portanto seleciono button para vagas

In [ ]:
sleep(3)
driver.find_elements(by=By.XPATH, value='/html/body/nav/ul/li[4]/a')[0].click()

###### Em seguida, limpo a label de localidade e a label de palavras-chaves para vagas, e insiro a que quero buscar

In [ ]:
driver.find_element('name','keywords').clear()
driver.find_element('name','keywords').send_keys('Marketing E Publicidade')
driver.find_element('name','location').clear()
driver.find_element('name','location').send_keys('Brasil')

###### Seleciono o localidade 1 das opções que aparece ao preencher a label

In [ ]:
sleep(3)
driver.find_elements(by=By.XPATH, value='//*[@id="location-1"]')[0].click()

###### Aqui irei abrir os filtros para tipos de vaga, e selecionar o filtro Tempo integral e Estágio, por último seleciono o botão concluído

In [ ]:
sleep(3)
driver.find_elements(by=By.XPATH, value='//*[@id="jserp-filters"]/ul/li[4]/div/div/button')[0].click()

In [ ]:
sleep(3)
driver.find_elements(by=By.XPATH, value='//*[@id="jserp-filters"]/ul/li[4]/div/div/div/div/div/div[1]/label')[0].click()

In [ ]:
'''sleep(3)
driver.find_elements(by=By.XPATH, value='//*[@id="jserp-filters"]/ul/li[4]/div/div/div/div/div/div[5]/label')[0].click()'''

In [ ]:
sleep(3)
driver.find_elements(by=By.XPATH, value='//*[@id="jserp-filters"]/ul/li[4]/div/div/div/button')[0].click()

In [ ]:
# fecho evento de iniciar sessão
try:
    evento_sessao = driver.find_elements(by=By.XPATH, value='/html/body/div[3]/button')[0]
    # Se o elemento estiver presente, clica nele para fechar o evento
    evento_sessao.click()
except:
    # Se o elemento não estiver presente, não faz nada
    pass

###### Pegando link das vagas

In [ ]:
'''# Realizar o scrolling até o fim da página
contador = 0
altura_anterior = driver.execute_script("return window.pageYOffset;")
while contador < 2:
    # Realizar o scrolling até o fim da página
    actions = ActionChains(driver)
    actions.send_keys(Keys.END).perform()
    sleep(2)  # Aguardar um tempo para carregar mais vagas
    
    # Verificar se não há mais vagas para carregar (chegou ao fim da página)
    try:
        even_ver_mais = driver.find_elements(by=By.XPATH, value='//*[@id="main-content"]/section[2]/button')[0]
        #even_ver_mais.click()
        driver.execute_script("arguments[0].click();", even_ver_mais)
        # Verificar se a altura da página foi alterada
        altura_atual = driver.execute_script("return window.pageYOffset;")
        if altura_atual == altura_anterior:
            # Incrementar o contador quando o botão não estiver mais disponível
            contador += 1
        else:
            # Zerar o contador quando a página for expandida
            contador = 0
        altura_anterior = altura_atual
    except NoSuchElementException:
         # Incrementar o contador quando o botão não estiver mais disponível
        contador += 1
        continue'''

vagas_links = driver.find_elements(By.XPATH, '/html/body/div[1]/div/main/section[2]/ul/li/div/a')
vagas_links = [link.get_attribute('href') for link in vagas_links]
    
'''print(vagas_link)
print(len(vagas_links))'''

###### Pegando titulos das vagas

In [ ]:
vagas_title = driver.find_elements(by=By.CLASS_NAME, value='base-search-card__title')
vagas_title=[titulo.text for titulo in vagas_title]

'''print(vagas_title)
print(len(vagas_title_texts))'''

###### Pegando nome das empresas das vagas

In [ ]:
vagas_empresa = driver.find_elements(by=By.CLASS_NAME, value='hidden-nested-link')
vagas_empresa = [empresa.text for empresa in vagas_empresa]

'''print(vagas_empresa)
print()
print(len(vagas_empresa))'''

###### Pegando link das empresas das vagas e os dados de tipo e nivel de experiência

In [ ]:
def rasp(tipo, nivel):
    vagas_nivel = driver.find_elements(by=By.XPATH, value='//*[@id="main-content"]/section[1]/div/div/section[1]/div/ul/li[1]/span')
    for i in vagas_nivel:
        nivel.append(i.text)
    
    vagas_tipo = driver.find_elements(by=By.XPATH, value='//*[@id="main-content"]/section[1]/div/div/section[1]/div/ul/li[2]/span')
    for i in vagas_tipo:
        tipo.append(i.text)

In [ ]:
vagas_tipo = []
vagas_nivel = []

links_sites_emp = []

j = 1
for i in vagas_link:
    driver.get(i)
    
    #verificar se o url aberto foi o certo
    while driver.current_url != i:
        driver.get(i)
        #print(j,driver.current_url)
    
    rasp(vagas_nivel, vagas_tipo)
    
    site_empresa = driver.find_elements(by=By.XPATH, value='//*[@id="main-content"]/section[1]/div/section[2]/div/div[1]/div/h4/div[1]/span[1]/a')
    for k in site_empresa:
        print(k.get_attribute('href'))
        links_sites_emp.append(k.get_attribute('href'))
    j = j + 1
    

In [ ]:
def rasp_info_emp(seguidores_emp, local_emp, qtd_funcionarios):
    #tratar erro de quando não encontrar informações de texto sobre seguidores no linkedin
    try:
        elements = driver.find_elements(by=By.XPATH, value='//*[@id="main-content"]/section[1]/section/div/div[2]/div[1]/h3')
        if len(elements) > 0:
            info_aux = re.split(padrao, elements[0].text, maxsplit = 1)
            seguidores_emp.append(info_aux[1]+info_aux[2])
        else:
            seguidores_emp.append("Não informado")

    except NoSuchElementException:
        seguidores_emp.append("Não informado")


    element_loc = driver.find_elements(by=By.XPATH, value='//*[@id="address-0"]')
    local_emp.append(element_loc[0].text)
    

    element_funcionario = driver.find_elements(by=By.XPATH, value='//*[@id="main-content"]/section[1]/div/section[1]/div/dl/div[3]/dd')
    qtd_funcionarios.append(element_funcionario[0].text)

In [ ]:
seguidores_empresa, local_empresa, qtd_func = [], [], []
#para separar a string quando encontrar o primeiro digito
padrao = r"(\d)" #significa um digito

for i in links_sites_emp:
    driver.get(i)
    print(driver.current_url)

    #verificar se o url aberto foi o certo
    while driver.current_url != i:
        driver.get(i)
        #print(j,driver.current_url)

    sleep(3)

    # fecho evento de logar se estiver presente
    try:
        evento_entrar = driver.find_elements(by=By.XPATH, value='//*[@id="organization_guest_contextual-sign-in"]/div/section/button')[0]
        # Se o elemento estiver presente, clica nele para fechar o evento
        evento_entrar.click()
    except:
        # Se o elemento não estiver presente, não faz nada
        pass
    
    rasp_info_emp(seguidores_empresa, local_empresa, qtd_func)


'''print(seguidores_empresa)
print(local_empresa)
print(qtd_func)
print(len(seguidores_empresa), len(local_empresa), len(qtd_func))'''